# Logging and Evaluation your Customer Support Agent using Maxim SDK

Welcome to the tutorial on how to use the [Maxim python SDK](https://www.getmaxim.ai/docs/sdk/observability/python/overview) to easily log and evaluate your Customer Support agent using decorators.

We hope you have read the guide [here](https://github.com/maximhq/maxim-cookbooks/tree/main/python/simulation/simulation_workflow.md) and are familiar with the basic structure of the agent.

Let's head on to the logging and evaluation. 

##### **Learning Objectives**

By the end of this tutorial, you will be able to:
- Understand the basics of tracing using Maxim SDK.
- Use the SDK to trace your customer support agent.
- Evaluate the performance of your agent using Maxim's built-in evaluators.

## Setting Up

### Necessary Imports

In [ ]:
import openai
from uuid import uuid4
from fastapi import FastAPI
from pydantic import BaseModel
from typing import Dict, List, Optional

from src.llm_agent import LLMAgent
from src.embeddings import EmbeddingManager

from maxim import Maxim, Config
from maxim.logger import LoggerConfig, SessionConfig
from maxim.decorators import trace, current_trace, retrieval, current_retrieval, generation, current_generation

### Configuring Objects

For the purpose of this tutorial and modular systems, distributed tracing might be good to understand, refer to this for the same: https://www.getmaxim.ai/docs/observability/quickstart/distributed-tracing

In [ ]:
maxim_api_key = "<your-maxim-api-key>"
maxim_log_repository_id = "<your-maxim-log-repository-id>"

maxim = Maxim(Config(api_key=maxim_api_key))
logger = maxim.logger(LoggerConfig(id=maxim_log_repository_id))

agent = LLMAgent()
app = FastAPI()

### Agent Endpoint
Learn more about it here: https://www.getmaxim.ai/docs/observability/concepts#trace

In our implementation of the customer support agent, we have a `/chat` endpoint that is used to recieve user input and post processing from the agent, send the response back to the user. 

A few things to note:
- The session ID is the same as the conversation ID, which is either sent along with the request or is randomly generated during runtime.
- A session captures an entire conversation, while a trace focuses more on a single complete turn of the conversation. You can learn more about them here: https://www.getmaxim.ai/docs/observability/concepts

In [ ]:
class ChatRequest(BaseModel):
    query: str
    conversation_id: Optional[str] = None

@trace(logger=logger, id=str(uuid4()), sessionId=lambda request: request.conversation_id, name="chat-endpoint", evaluators=['toxicity', 'bias'])
@app.post('/chat')
async def chat(
    request: ChatRequest,
) -> Dict:
    """
    Endpoint to handle chat requests.
    """
    # Defining variables
    conversation_id = request.conversation_id or str(uuid4())
    query = request.query

    # Initializing Session
    session_name = f"Session ID: {conversation_id}"
    session = logger.session(SessionConfig(id=conversation_id, name=session_name))

    current_trace().set_input(query)
    # Calling the main agent function
    response = agent.customer_support_pipeline(query, conversation_id)
    current_trace().set_output(response)
    current_trace().evaluate().with_variables({'output': response.content}, ['toxicity', 'bias'])
    
    # Once completed, we can end the trace, session and cleanup the logger
    current_trace().end()
    session.end()
    logger.cleanup()

    return response

## Capturing Components
Learn more about retrieval here: https://www.getmaxim.ai/docs/observability/concepts#retrieval

### Retrieval Context

A customer support agent requires a knowledge base for context specific answers. The knowledge base in this case are the customer, product and order details, along with policy documents.
The agents needs to be able to retrieve the relevant sections of the base in order to answer user queries effectively.
Here is how it works:

In [ ]:
class LLMAgent:
   @retrieval(logger=logger, id=str(uuid4()), name="knowledge-retrieval", evaluators=["context relevance", "output relevance"])
   def customer_support_pipeline(
        self, 
        user_query: str,
        conversation_id: str,
        num_chunks: int = 3
    ) -> List[Dict]:
        """Complete support pipeline with conversation persistence."""

        # Get response from retrieval pipeline (returns a list of tuples)
        retrieved_chunks = EmbeddingManager.search(user_query, k=num_chunks)

        current_retrieval().input(user_query)
        retrieved_chunks_log = [str(chunk) for chunk in retrieved_chunks_log]
        current_retrieval().output(retrieved_chunks_log)
        
        retrieved_chunks_log = "\n\n".join(retrieved_chunks)
        current_retrieval().evaluate().with_variables({'input': user_query, 'context': retrieved_chunks_log}, ['context relevance'])

        message_history = None       
        # load conversation into message_history using conversation_id for example     
        # generate response

        latest_response = self.generate_response(message_history)
        current_retrieval().evaluate().with_variables({'input': user_query, 'output': latest_response}, ['output relevance'])
        current_retrieval().end()

        # save conversation
        return latest_response

### LLM generation
Learn more about generation here: https://www.getmaxim.ai/docs/observability/concepts#generation

One of the most important things to log is the LLM generation to a user query. 
Using generation decorator, we can also evaluate the agent's performance and identify issues underlying the LLM's behavior. 

A few things to keep in mind:
- While we in this example use OpenAI, you can use any LLM model of choice that supports function calls.
- These are code templates to help out, implementation often varies from system to system.

In [ ]:
class LLMAgent:
    @generation(logger=logger, id=str(uuid4()), name="openai-generation", evaluators=['toxicity', 'bias'])
    def generate_response(
            self, 
            messages: List[Dict],
        ) -> List[Dict]:
            """Generate a response using GPT-4 with retrieved context and handle tool calls."""

            # Initialize the generation (Learn more about generation here: )
            current_generation().provider("openai")
            current_generation().model("gpt-4o")
            # obtain a response from the model (remember to pass the tools)
            response = openai.chat.completions.create()

            current_generation().result(response)
            # Attaching evaluators to the generation
            current_generation().evaluate().with_variables({'output': response.content}, ['toxicity', 'bias'])
            current_generation().end()


            return response

### Tool Calls
Learn more about tool call here: https://www.getmaxim.ai/docs/observability/concepts#tool-call

Another important part of any agent's work is the tool calls being made. These are the critical points where most agents tend to fail, logging them and being able evaluate comes as a big part of the improvement process.

In [ ]:
# continuing from the previous codeblock
class LLMAgent:
    @tool_call(logger=logger, id=str(uuid4()), name="tool_call_name", description="tool_call_description", args="tool_call_args", evaluators=['toxicity', 'bias'])
    def generate_response(
            self, 
            messages: List[Dict],
        ) -> List[Dict]:
            """Generate a response using GPT-4 with retrieved context and handle tool calls."""
            
            response = openai.chat.completions.create()

            # Initialize the tool call configuration
            tool_call = response.tool_calls[0]
            tool_call_name = tool_call.function.name
            tool_call_args = tool_call.function.arguments
            tool_call_description = "" # Place a tool call description here

            # Perform tool calling as necessary
            tool_result = self.execute_tool(tool_call)

            # Log the tool call result obtained
            current_tool_call().result(tool_result)
            current_tool_call().end()

            return tool_result

## Conclusion

Congratulations on reaching the end of this guide! 🚀 By now, you should have a solid integration of the Maxim SDK into your project and should start seeing the results on the platform in your log repository.
To learn more about the Maxim SDK, checkout the Maxim SDK documentation [here](https://www.getmaxim.ai/docs/sdk/overview).

Happy tracing!